In [3]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from typing import Callable, List, Optional, Dict, Union

from evidently.test_preset import DataDriftTestPreset
from evidently.test_suite import TestSuite
from tqdm import tqdm

from helpers import predict, load_agent, quality, clip
from preprocess import preprocess_stats
from rl.sim_enviroment import SimulatedCustomEnv

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.options import DataDriftOptions
from drift_calculator import DriftCalculator

/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/si

In [4]:
train_df = pd.read_csv('data/train_2020-2023.csv', )
test_df = pd.read_csv('data/test_2020-2023.csv', )

In [5]:
train_df

,DATA,Cell ID,Number of Available\nTCH,HR Usage Rate,"TCH Blocking Rate, BH","TCH Traffic (Erl), BH",Param 1,Param 2
0,2020-10-02,25771,4.0,94.95,0.0,7.78,56,68
1,2020-10-03,25771,4.0,98.02,0.0,8.85,56,68
2,2020-10-04,25771,4.0,96.86,0.0,6.35,56,68
3,2020-10-05,25771,4.0,95.82,0.0,8.82,56,68
4,2020-10-06,25771,4.0,97.75,0.0,8.13,56,68
...,...,...,...,...,...,...,...,...
838163,2023-03-10,14047,28.0,34.00,0.0,4.24,12,33
838164,2023-03-11,14047,28.0,34.00,0.0,4.24,12,33
838165,2023-03-12,14047,28.0,58.00,0.0,4.34,12,33
838166,2023-03-13,14047,28.0,58.00,0.0,4.34,12,33


In [6]:
test_df

,DATA,Cell ID,Number of Available\nTCH,HR Usage Rate,"TCH Blocking Rate, BH","TCH Traffic (Erl), BH",Param 1,Param 2
0,2023-02-03,25771,13.0,93.0,0.0,3.88,46,54
1,2023-02-04,25771,13.0,93.0,0.0,3.88,46,54
2,2023-02-05,25771,13.0,98.0,0.0,4.89,46,54
3,2023-02-06,25771,13.0,100.0,0.0,4.87,46,54
4,2023-02-07,25771,12.0,100.0,0.0,8.89,46,54
...,...,...,...,...,...,...,...,...
210030,2023-08-31,14047,28.0,34.0,0.0,8.36,12,33
210031,2023-09-01,14047,28.0,49.0,0.0,6.81,12,33
210032,2023-09-02,14047,28.0,41.0,0.0,7.07,12,33
210033,2023-09-03,14047,28.0,41.0,0.0,7.07,12,33


In [6]:
test_df.index

RangeIndex(start=0, stop=210035, step=1)

In [7]:
len(train_df['Cell ID'].unique()), len(set(train_df['Cell ID'].unique()).intersection(test_df['Cell ID'].unique()))

(994, 994)

In [7]:
stat_tests = [
    'ks', # <= 1000 Kolmogorov–Smirnov
    'wasserstein', # > 1000 Wasserstein distance (normed)
    'kl_div', # Kullback-Leibler divergence
    'psi', # Population Stability Index
    'jensenshannon',  #  > 1000 Jensen-Shannon distance
    # 'anderson', # Anderson-Darling test
    'cramer_von_mises', # Cramer-Von-Mises test
    'hellinger', # Hellinger Distance (normed)
    'mannw', # Mann-Whitney U-rank test
    'ed', # Energy distance
    # 'es', # Epps-Singleton tes
    't_test', # T-Test
    'emperical_mmd', # Emperical-MMD
]

In [38]:
save_path = 'data/generated/drift/by_cell_agent/run_1/'
Path(save_path).mkdir(exist_ok=True, parents=True)

drift_calc = DriftCalculator([None] + stat_tests, ['default'] + stat_tests)
drift_scores = []
cols = ['Number of Available\nTCH', 'HR Usage Rate', 'TCH Blocking Rate, BH',
       'TCH Traffic (Erl), BH', 'Param 1', 'Param 2']
ref = train_df

for cell in tqdm(train_df['Cell ID'].value_counts().keys()[:]):
    # ref = train_df[train_df['Cell ID'] == cell]
    cur = test_df[test_df['Cell ID'] == cell]
    # add original distribution
    # cur = pd.concat([ref.sample(n=len(ref) - len(cur)), cur])

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        score = drift_calc.get_drift(current_data=cur, reference_data=ref, sample=True, weighted=False, save_plot=f'{save_path}cell_{cell}')

    score['cell_id'] = cell
    drift_scores.append(score)

drift_scores_df = pd.DataFrame(drift_scores)
drift_scores_df.to_csv(f'{save_path}/by_cell_sampled.csv')
drift_scores_df

100%|██████████| 994/994 [4:24:05<00:00, 15.94s/it]  


,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,mannw,ed,t_test,emperical_mmd,cell_id
0,0.875,0.875,0.875,0.875,1.000,1.000,0.875,1.000,0.750,0.875,0.875,1.0,25771
1,0.875,0.875,0.875,0.875,1.000,0.875,1.000,1.000,0.875,0.875,0.875,1.0,22944
2,1.000,1.000,1.000,1.000,1.000,0.875,1.000,0.875,0.750,1.000,0.875,1.0,26335
3,1.000,1.000,1.000,0.875,1.000,0.875,1.000,0.875,0.375,1.000,1.000,1.0,26332
4,0.875,0.875,1.000,1.000,1.000,0.875,0.875,0.875,0.625,0.875,0.875,1.0,26336
...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,0.875,0.875,0.875,0.875,0.875,0.875,1.000,0.875,0.125,0.875,1.000,1.0,5683
990,0.875,0.875,0.875,1.000,1.000,0.875,0.875,0.875,0.250,0.875,0.750,1.0,5682
991,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.875,0.875,1.000,0.875,1.0,12772
992,0.875,0.875,1.000,1.000,1.000,1.000,0.875,0.875,0.625,0.875,0.875,1.0,12771


In [39]:
drift_scores_df.describe()

,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,mannw,ed,t_test,emperical_mmd,cell_id
count,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000
mean,0.914487,0.914235,0.966549,0.937752,0.955734,0.943662,0.945423,0.913481,0.437626,0.935614,0.855257,0.973717,14601.775654
std,0.059211,0.059378,0.055367,0.063032,0.059811,0.062227,0.062280,0.057729,0.200409,0.062503,0.094556,0.051270,11994.634963
min,0.750000,0.750000,0.875000,0.750000,0.875000,0.875000,0.750000,0.875000,0.125000,0.875000,0.500000,0.750000,701.000000
25%,0.875000,0.875000,0.875000,0.875000,0.875000,0.875000,0.875000,0.875000,0.250000,0.875000,0.750000,1.000000,5071.250000
50%,0.875000,0.875000,1.000000,1.000000,1.000000,1.000000,1.000000,0.875000,0.375000,0.875000,0.875000,1.000000,10931.500000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.625000,1.000000,0.875000,1.000000,24234.500000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,42857.000000


In [40]:
rewards = pd.read_csv('data/generated/drift/by_cell_agent/drift_scores_rewards_new_agent_train-test_no_sample20-23.csv', index_col=0)
rewards

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
0,10932,0.875,0.991378,0.973693,0.994549,0.002269,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
1,12762,0.875,0.986723,0.790479,0.992018,0.019664,1010.000000,2040.0,598.999165,9.855072,-20.0,10.0,2.085144
2,12781,0.875,0.987593,0.949088,0.992366,0.004489,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
3,12782,0.875,0.986304,0.972973,0.992206,0.003253,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
4,12783,1.000,0.949899,0.779899,0.991859,0.049240,925.797101,1920.0,558.289010,9.275362,-20.0,10.0,4.617035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.875,0.992266,0.967622,0.993875,0.002159,800.000000,1590.0,460.434577,10.000000,10.0,10.0,0.000000
990,5682,0.875,0.991784,0.980978,0.994072,0.001335,800.000000,1590.0,460.434577,10.000000,10.0,10.0,0.000000
991,12771,0.875,0.990560,0.978193,0.994569,0.002849,570.000000,1130.0,327.643099,10.000000,10.0,10.0,0.000000
992,12772,1.000,0.968540,0.751103,0.986172,0.028634,550.884956,1100.0,315.678288,9.734513,-20.0,10.0,2.822163


In [41]:
merged = rewards.merge(drift_scores_df, left_on='cell_id', right_on='cell_id', how='inner')
merged

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,mannw,ed,t_test,emperical_mmd
0,10932,0.875,0.991378,0.973693,0.994549,0.002269,1040.000000,2070.0,598.999165,10.000000,...,0.875,0.875,0.875,1.000,0.875,0.875,0.250,0.875,0.875,1.000
1,12762,0.875,0.986723,0.790479,0.992018,0.019664,1010.000000,2040.0,598.999165,9.855072,...,1.000,1.000,1.000,1.000,0.875,1.000,0.375,0.875,0.750,0.875
2,12781,0.875,0.987593,0.949088,0.992366,0.004489,1040.000000,2070.0,598.999165,10.000000,...,0.875,0.875,0.875,1.000,0.875,0.875,0.625,0.875,0.750,0.875
3,12782,0.875,0.986304,0.972973,0.992206,0.003253,1040.000000,2070.0,598.999165,10.000000,...,1.000,0.875,1.000,1.000,1.000,0.875,0.875,1.000,0.625,1.000
4,12783,1.000,0.949899,0.779899,0.991859,0.049240,925.797101,1920.0,558.289010,9.275362,...,1.000,1.000,1.000,1.000,1.000,1.000,0.750,1.000,1.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.875,0.992266,0.967622,0.993875,0.002159,800.000000,1590.0,460.434577,10.000000,...,0.875,0.875,0.875,0.875,1.000,0.875,0.125,0.875,1.000,1.000
990,5682,0.875,0.991784,0.980978,0.994072,0.001335,800.000000,1590.0,460.434577,10.000000,...,0.875,1.000,1.000,0.875,0.875,0.875,0.250,0.875,0.750,1.000
991,12771,0.875,0.990560,0.978193,0.994569,0.002849,570.000000,1130.0,327.643099,10.000000,...,1.000,1.000,1.000,1.000,0.875,0.875,0.625,0.875,0.875,1.000
992,12772,1.000,0.968540,0.751103,0.986172,0.028634,550.884956,1100.0,315.678288,9.734513,...,1.000,1.000,1.000,1.000,1.000,0.875,0.875,1.000,0.875,1.000


In [42]:
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,mannw,ed,t_test,emperical_mmd
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.042598,0.047759,-0.009873,0.092648,-0.031061,-0.087253,0.230481,-0.022748,-0.015057,0.026413
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,0.303737,-0.013650,0.198249,0.127772,0.273662,0.301350,0.316115,0.603874,0.136864,0.187260
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.202910,-0.030971,-0.173745,-0.196127,-0.175496,-0.343140,-0.382826,-0.421578,-0.167796,-0.133152
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.115599,0.060393,-0.163994,-0.150701,0.003275,-0.271364,-0.187097,-0.348418,-0.069603,0.012540
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.098021,-0.063517,-0.049393,-0.097775,-0.133379,-0.166729,-0.538084,-0.239325,-0.116631,-0.107598
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.160203,-0.042423,0.199975,0.182354,0.023793,0.332900,0.192295,0.381977,0.110849,0.030908
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.143673,-0.029941,-0.177776,-0.161609,-0.020517,-0.254315,-0.116398,-0.245121,-0.118458,-0.063217
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.141057,-0.053018,-0.184300,-0.166422,-0.022861,-0.234642,-0.104634,-0.223359,-0.116993,-0.077174
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.097537,-0.092611,-0.147571,-0.132495,-0.032062,-0.133021,-0.061584,-0.116163,-0.086605,-0.090112
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.130821,0.005358,-0.160916,-0.148733,-0.014145,-0.272751,-0.130990,-0.258347,-0.119445,-0.062657


In [37]:
merged[merged['wasserstein'] != merged['drift_score']]

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,mannw,ed,t_test,emperical_mmd
0,10932,0.875,0.991378,0.973693,0.994549,0.002269,1040.0,2070.0,598.999165,10.0,...,1.0,0.875,0.875,0.875,0.875,0.875,0.375,0.875,0.625,1.000
3,12782,0.875,0.986304,0.972973,0.992206,0.003253,1040.0,2070.0,598.999165,10.0,...,1.0,0.875,1.000,0.875,1.000,0.875,0.875,1.000,0.625,1.000
5,13301,0.875,0.991404,0.985357,0.995448,0.001955,1040.0,2070.0,598.999165,10.0,...,1.0,1.000,1.000,1.000,1.000,0.875,0.500,0.875,0.750,1.000
6,13302,0.875,0.985855,0.978636,0.991583,0.002310,1040.0,2070.0,598.999165,10.0,...,1.0,1.000,0.875,1.000,1.000,0.875,0.875,0.875,0.750,1.000
9,13313,0.875,0.987173,0.972817,0.992569,0.003328,1040.0,2070.0,598.999165,10.0,...,1.0,0.875,0.875,0.875,1.000,1.000,1.000,1.000,0.875,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,26931,0.875,0.988672,0.915147,0.994018,0.009859,810.0,1610.0,466.208108,10.0,...,1.0,1.000,1.000,1.000,0.875,0.875,0.625,0.875,0.750,0.875
989,5683,0.875,0.992266,0.967622,0.993875,0.002159,800.0,1590.0,460.434577,10.0,...,1.0,1.000,1.000,1.000,1.000,0.875,0.125,0.875,0.875,1.000
990,5682,0.875,0.991784,0.980978,0.994072,0.001335,800.0,1590.0,460.434577,10.0,...,1.0,0.875,1.000,1.000,0.875,0.875,0.125,0.875,0.875,1.000
991,12771,0.875,0.990560,0.978193,0.994569,0.002849,570.0,1130.0,327.643099,10.0,...,1.0,1.000,1.000,1.000,1.000,1.000,0.625,0.875,0.625,1.000


## No sample | evidently 0.4.5

In [33]:
stat_tests = [
    'ks', # <= 1000 Kolmogorov–Smirnov
    'wasserstein', # > 1000 Wasserstein distance (normed)
    'kl_div', # Kullback-Leibler divergence
    'psi', # Population Stability Index
    'jensenshannon',  #  > 1000 Jensen-Shannon distance
    # # 'anderson', # Anderson-Darling test
    'cramer_von_mises', # Cramer-Von-Mises test
    'hellinger', # Hellinger Distance (normed)
    # 'mannw', # Mann-Whitney U-rank test (too long ~23 s on iteration)
    'ed', # Energy distance
    # # 'es', # Epps-Singleton tes
    't_test', # T-Test
    # 'emperical_mmd', # Emperical-MMD (takes too much space to compute on sampled data)
]

In [34]:
save_path = 'data/generated/drift/by_cell_agent/run_2/'
Path(save_path).mkdir(exist_ok=True, parents=True)

drift_calc = DriftCalculator([None] + stat_tests , ['default'] + stat_tests )
drift_scores = []
cols = ['Number of Available\nTCH', 'HR Usage Rate', 'TCH Blocking Rate, BH',
       'TCH Traffic (Erl), BH', 'Param 1', 'Param 2']
ref = train_df

for cell in tqdm(train_df['Cell ID'].value_counts().keys()[:]):
    # ref = train_df[train_df['Cell ID'] == cell]
    cur = test_df[test_df['Cell ID'] == cell]
    # add original distribution
    # cur = pd.concat([ref.sample(n=len(ref) - len(cur)), cur])

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        score = drift_calc.get_drift(current_data=cur, reference_data=ref, sample=False, weighted=False, save_plot=None)
        # data_drift_report = Report(metrics=[
        #     DataDriftPreset(),
        # ])
        # data_drift_report.run(reference_data=ref, current_data=cur,)
        # drift = data_drift_report.as_dict()['metrics'][0]['result']['share_of_drifted_columns']
        # score = {'default': drift}

    score['cell_id'] = cell
    drift_scores.append(score)

drift_scores_df = pd.DataFrame(drift_scores)
drift_scores_df.to_csv(f'{save_path}/by_cell_not-sampled.csv')
drift_scores_df

  0%|          | 1/994 [02:02<33:54:34, 122.94s/it]


KeyboardInterrupt: 

In [32]:
import time

for t in stat_tests:
    start_time = time.time()

    data_drift_report = Report(metrics=[
        DataDriftPreset(num_stattest=t),
    ])
    data_drift_report.run(reference_data=ref, current_data=cur,)
    drift = data_drift_report.as_dict()['metrics'][0]['result']['share_of_drifted_columns']

    print(f'{t} : {(time.time() - start_time) } s')

ks : 9.86045241355896 s
wasserstein : 10.478918313980103 s
kl_div : 9.536772727966309 s
psi : 9.305282592773438 s
jensenshannon : 9.349831819534302 s
cramer_von_mises : 12.657140970230103 s
hellinger : 8.97890591621399 s
mannw : 23.36404061317444 s
ed : 10.078694343566895 s


/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/rid/Soft/anaconda3/envs/sm_bach

t_test : 8.555386781692505 s


In [14]:
rewards = pd.read_csv('data/generated/drift/by_cell_agent/drift_scores_rewards_new_agent_train-test_no_sample20-23.csv', index_col=0)
rewards

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
0,10932,0.875,0.991378,0.973693,0.994549,0.002269,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
1,12762,0.875,0.986723,0.790479,0.992018,0.019664,1010.000000,2040.0,598.999165,9.855072,-20.0,10.0,2.085144
2,12781,0.875,0.987593,0.949088,0.992366,0.004489,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
3,12782,0.875,0.986304,0.972973,0.992206,0.003253,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
4,12783,1.000,0.949899,0.779899,0.991859,0.049240,925.797101,1920.0,558.289010,9.275362,-20.0,10.0,4.617035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.875,0.992266,0.967622,0.993875,0.002159,800.000000,1590.0,460.434577,10.000000,10.0,10.0,0.000000
990,5682,0.875,0.991784,0.980978,0.994072,0.001335,800.000000,1590.0,460.434577,10.000000,10.0,10.0,0.000000
991,12771,0.875,0.990560,0.978193,0.994569,0.002849,570.000000,1130.0,327.643099,10.000000,10.0,10.0,0.000000
992,12772,1.000,0.968540,0.751103,0.986172,0.028634,550.884956,1100.0,315.678288,9.734513,-20.0,10.0,2.822163


In [15]:
merged = rewards.merge(drift_scores_df, left_on='cell_id', right_on='cell_id', how='inner')
merged

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std,default
0,22961,0.875,0.991694,0.989737,0.994441,0.000679,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875
1,22962,0.875,0.991775,0.965787,0.995182,0.003246,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875
2,22963,0.875,0.991169,0.965944,0.994598,0.002697,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875
3,22945,0.875,0.986486,0.944031,0.992333,0.003328,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875
4,22944,0.875,0.983747,0.977482,0.988741,0.001965,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875
5,22941,0.875,0.990761,0.854121,0.993822,0.010361,1031.014493,2040.000000,588.130521,9.855072,-20.000000,10.0,2.085144,0.875
6,22942,0.875,0.979845,0.887228,0.988778,0.011019,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875
7,22943,0.875,0.990964,0.961579,0.993207,0.002507,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875
8,26621,0.875,0.986950,0.966421,0.993065,0.003787,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875
9,26623,0.875,0.983032,0.955352,0.990751,0.005477,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000,0.875


In [16]:
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std,default
cell_id,1.000000,0.104289,-0.018947,0.140996,-0.200120,-0.121292,-0.016449,0.058901,0.146776,0.058901,0.097771,NaN,-0.081296,0.104289
drift_score,0.104289,1.000000,-0.598600,-0.506046,-0.136239,0.631725,-0.463077,-0.405270,-0.261500,-0.405270,-0.315002,NaN,0.359642,1.000000
quality_avg,-0.018947,-0.598600,1.000000,0.640644,0.702638,-0.718492,0.556996,0.516535,0.393531,0.516535,0.491353,NaN,-0.508556,-0.598600
quality_min,0.140996,-0.506046,0.640644,1.000000,0.099364,-0.973964,0.773525,0.867426,0.856593,0.867426,0.843444,NaN,-0.864247,-0.506046
quality_max,-0.200120,-0.136239,0.702638,0.099364,1.000000,-0.133902,0.119283,0.061642,0.007487,0.061642,0.098315,NaN,-0.082843,-0.136239
quality_std,-0.121292,0.631725,-0.718492,-0.973964,-0.133902,1.000000,-0.795757,-0.853717,-0.777604,-0.853717,-0.788490,NaN,0.827348,0.631725
cum_reward_avg,-0.016449,-0.463077,0.556996,0.773525,0.119283,-0.795757,1.000000,0.960050,0.755300,0.960050,0.897601,NaN,-0.936484,-0.463077
cum_reward_max,0.058901,-0.405270,0.516535,0.867426,0.061642,-0.853717,0.960050,1.000000,0.892583,1.000000,0.953231,NaN,-0.985659,-0.405270
cum_reward_std,0.146776,-0.261500,0.393531,0.856593,0.007487,-0.777604,0.755300,0.892583,1.000000,0.892583,0.903331,NaN,-0.909091,-0.261500
mom_reward_avg,0.058901,-0.405270,0.516535,0.867426,0.061642,-0.853717,0.960050,1.000000,0.892583,1.000000,0.953231,NaN,-0.985659,-0.405270
